In [7]:
# NHIỆM VỤ 4: MERGE DỮ LIỆU KHÁCH SẠN
import pandas as pd
from pathlib import Path

print("=" * 60)
print("NHIỆM VỤ 4: MERGE DỮ LIỆU KHÁCH SẠN")
print("=" * 60)

# ------------------------------------------------------------
# 1. THIẾT LẬP ĐƯỜNG DẪN & NHẬP XUẤT FILE
# ------------------------------------------------------------
print("\n1. THIẾT LẬP ĐƯỜNG DẪN & NHẬP XUẤT FILE")
print("-" * 50)

# Lấy thư mục hiện tại
BASE_DIR = Path.cwd()
SOURCE_DIR = BASE_DIR  # File CSV nằm cùng thư mục
OUTPUT_DIR = BASE_DIR / "data_clean"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Thư mục làm việc: {BASE_DIR}")
print(f"Đọc dữ liệu từ: {SOURCE_DIR}")
print(f"Xuất dữ liệu đến: {OUTPUT_DIR}")

# Kiểm tra file tồn tại
required_files = ['hotel_info.csv', 'booking_records.csv', 'guest_review.csv']
missing_files = []

for file in required_files:
    file_path = SOURCE_DIR / file
    if file_path.exists():
        print(f" File {file} tồn tại")
    else:
        print(f" File {file} không tồn tại")
        missing_files.append(file)

if missing_files:
    print(f"Thiếu file: {', '.join(missing_files)}")
    exit()

# Đọc file CSV
hotel_info = pd.read_csv('hotel_info.csv')
booking_records = pd.read_csv('booking_records.csv')
guest_review = pd.read_csv('guest_review.csv')

print(f"\nĐã đọc hotel_info.csv: {hotel_info.shape[0]} dòng, {hotel_info.shape[1]} cột")
print(f"Đã đọc booking_records.csv: {booking_records.shape[0]} dòng, {booking_records.shape[1]} cột")
print(f"Đã đọc guest_review.csv: {guest_review.shape[0]} dòng, {guest_review.shape[1]} cột")

# ------------------------------------------------------------
# 2. LÀM SẠCH DỮ LIỆU
# ------------------------------------------------------------
print("\n" + "="*50)
print("2. LÀM SẠCH DỮ LIỆU")
print("="*50)

# 2.1. Làm sạch hotel_info
print("\n2.1. Làm sạch hotel_info.csv")

hotel_info_clean = hotel_info.copy()

# Sử dụng str.strip() và str.upper()
hotel_info_clean['hotel_id'] = hotel_info_clean['hotel_id'].astype(str).str.strip().str.upper()
hotel_info_clean['hotel_name'] = hotel_info_clean['hotel_name'].astype(str).str.strip().str.title()

# Chuẩn hóa city
city_mapping = {
    'ha noi': 'Hà Nội', 'hn': 'Hà Nội',
    'da nang': 'Đà Nẵng', 'danang': 'Đà Nẵng',
    'tp.hcm': 'TP.HCM', 'tphcm': 'TP.HCM', 'tp hcm': 'TP.HCM',
    'hcm': 'TP.HCM', 'ho chi minh': 'TP.HCM'
}
hotel_info_clean['city'] = hotel_info_clean['city'].astype(str).str.strip().str.lower()
hotel_info_clean['city'] = hotel_info_clean['city'].map(city_mapping).fillna(hotel_info_clean['city'].str.title())

# Chuẩn hóa room_type
hotel_info_clean['room_type'] = hotel_info_clean['room_type'].astype(str).str.strip().str.title()

# Làm sạch base_price
hotel_info_clean['base_price'] = hotel_info_clean['base_price'].astype(str)
hotel_info_clean['base_price'] = hotel_info_clean['base_price'].str.replace(r'[^\d.]', '', regex=True)
hotel_info_clean['base_price'] = hotel_info_clean['base_price'].str.replace('.', '', regex=False)
hotel_info_clean['base_price'] = pd.to_numeric(hotel_info_clean['base_price'], errors='coerce')

# Sử dụng fillna() cho base_price
price_mean = hotel_info_clean['base_price'].mean()
hotel_info_clean['base_price'] = hotel_info_clean['base_price'].fillna(price_mean)
print(f"   Đã xử lý base_price: {hotel_info_clean['base_price'].isna().sum()} giá trị NaN")

print("   Đã làm sạch hotel_info")

# 2.2. Làm sạch booking_records
print("\n2.2. Làm sạch booking_records.csv")

booking_records_clean = booking_records.copy()

# Sử dụng str.strip() và str.upper()
booking_records_clean['booking_id'] = booking_records_clean['booking_id'].astype(str).str.strip().str.upper()
booking_records_clean['hotel_id'] = booking_records_clean['hotel_id'].astype(str).str.strip().str.upper()

# Sử dụng to_datetime()
booking_records_clean['checkin_date'] = pd.to_datetime(
    booking_records_clean['checkin_date'],
    errors='coerce',
    dayfirst=True
)
print(f"   Chuyển đổi checkin_date: {booking_records_clean['checkin_date'].isna().sum()} giá trị không hợp lệ")

# Làm sạch nights
booking_records_clean['nights'] = pd.to_numeric(booking_records_clean['nights'], errors='coerce')
invalid_nights = (booking_records_clean['nights'] <= 0).sum()
booking_records_clean.loc[booking_records_clean['nights'] <= 0, 'nights'] = None
booking_records_clean['nights'] = booking_records_clean['nights'].fillna(1).astype(int)
print(f"   Đã sửa {invalid_nights} giá trị nights không hợp lệ")

# Làm sạch num_guests
booking_records_clean['num_guests'] = pd.to_numeric(
    booking_records_clean['num_guests'].astype(str).str.strip(),
    errors='coerce'
)
invalid_guests = booking_records_clean['num_guests'].isna().sum()
booking_records_clean['num_guests'] = booking_records_clean['num_guests'].fillna(1).astype(int)
print(f"   Đã sửa {invalid_guests} giá trị num_guests không hợp lệ")

print("   Đã làm sạch booking_records")

# 2.3. Làm sạch guest_review
print("\n2.3. Làm sạch guest_review.csv")

guest_review_clean = guest_review.copy()

# Sử dụng str.strip() và str.upper()
guest_review_clean['booking_id'] = guest_review_clean['booking_id'].astype(str).str.strip().str.upper()

# Làm sạch rating - đảm bảo trong khoảng 1-5
guest_review_clean['rating'] = pd.to_numeric(guest_review_clean['rating'], errors='coerce')
invalid_ratings = ((guest_review_clean['rating'] < 1) | (guest_review_clean['rating'] > 5)).sum()
guest_review_clean.loc[(guest_review_clean['rating'] < 1) | (guest_review_clean['rating'] > 5), 'rating'] = None

# Sử dụng fillna() cho rating
rating_mean = guest_review_clean['rating'].mean()
guest_review_clean['rating'] = guest_review_clean['rating'].fillna(rating_mean)
print(f"   Đã sửa {invalid_ratings} giá trị rating không hợp lệ")

# Sử dụng fillna() cho comment
guest_review_clean['comment'] = guest_review_clean['comment'].fillna('').astype(str).str.strip()
empty_comments = (guest_review_clean['comment'] == '').sum()
print(f"   Có {empty_comments} comment trống")

print("   Đã làm sạch guest_review")

# ------------------------------------------------------------
# 3. XUẤT FILE ĐÃ LÀM SẠCH
# ------------------------------------------------------------
print("\n" + "="*50)
print("3. XUẤT FILE ĐÃ LÀM SẠCH")
print("="*50)

# Xuất file đã làm sạch
hotel_output = OUTPUT_DIR / "hotel_info_clean.csv"
booking_output = OUTPUT_DIR / "booking_records_clean.csv"
review_output = OUTPUT_DIR / "guest_review_clean.csv"

hotel_info_clean.to_csv(hotel_output, index=False)
booking_records_clean.to_csv(booking_output, index=False)
guest_review_clean.to_csv(review_output, index=False)

print(f" Đã xuất hotel_info_clean.csv: {hotel_output}")
print(f" Đã xuất booking_records_clean.csv: {booking_output}")
print(f" Đã xuất guest_review_clean.csv: {review_output}")

# ------------------------------------------------------------
# 4. TRUY VẤN DỮ LIỆU - Sử dụng Boolean Masking
# ------------------------------------------------------------
print("\n" + "="*50)
print("4. TRUY VẤN DỮ LIỆU TRƯỚC KHI MERGE")
print("="*50)

# 4.1. Kiểm tra booking không có hotel_info tương ứng
# Sử dụng isin() với Boolean Masking
hotel_ids_info = set(hotel_info_clean['hotel_id'])
missing_hotel_mask = ~booking_records_clean['hotel_id'].isin(hotel_ids_info)
missing_hotel_bookings = booking_records_clean[missing_hotel_mask]

print("4.1. Booking không có hotel_info tương ứng:")
print(f"   Số lượng: {len(missing_hotel_bookings)} booking")

# 4.2. Kiểm tra booking không có review
booking_ids_records = set(booking_records_clean['booking_id'])
booking_ids_review = set(guest_review_clean['booking_id'])
bookings_without_review = booking_ids_records - booking_ids_review

print("\n4.2. Booking không có review:")
print(f"   Số lượng: {len(bookings_without_review)} booking")

# 4.3. Sử dụng loc để xem chi tiết
if len(missing_hotel_bookings) > 0:
    print("\n4.3. Chi tiết booking sẽ bị mất:")
    print(missing_hotel_bookings.loc[:, ['booking_id', 'hotel_id']].head().to_string())

# ------------------------------------------------------------
# 5. MERGE DỮ LIỆU - Sử dụng merge() với inner/left
# ------------------------------------------------------------
print("\n" + "="*50)
print("5. MERGE DỮ LIỆU")
print("="*50)

# 5.1. Merge hotel_info với booking_records (INNER JOIN)
print("5.1. Merge hotel_info với booking_records (INNER JOIN):")

merged_step1 = pd.merge(
    hotel_info_clean,
    booking_records_clean,
    on='hotel_id',
    how='inner',
    suffixes=('_hotel', '_booking')
)

print(f"   Kết quả: {merged_step1.shape[0]} dòng")
print(f"   Số booking hợp lệ: {merged_step1['booking_id'].nunique()}")
print(f"   Số khách sạn có booking: {merged_step1['hotel_id'].nunique()}")

# 5.2. Merge với guest_review (LEFT JOIN)
print("\n5.2. Merge với guest_review (LEFT JOIN):")

final_merged = pd.merge(
    merged_step1,
    guest_review_clean,
    on='booking_id',
    how='left'
)

print(f"   Kết quả cuối: {final_merged.shape[0]} dòng, {final_merged.shape[1]} cột")
print(f"   Booking có rating: {final_merged['rating'].notna().sum()}")
print(f"   Booking không có rating: {final_merged['rating'].isna().sum()}")

# ------------------------------------------------------------
# 6. GROUPBY SAU MERGE - Sử dụng groupby() và agg()
# ------------------------------------------------------------
print("\n" + "="*50)
print("6. GROUPBY ĐỂ KIỂM TRA KẾT QUẢ MERGE")
print("="*50)

# 6.1. Số booking theo thành phố
# Sử dụng groupby() và size()
bookings_by_city = final_merged.groupby('city').size()
print("6.1. Số booking theo thành phố:")
for city, count in bookings_by_city.items():
    print(f"   {city}: {count} booking")

# 6.2. Thống kê theo loại phòng
# Sử dụng groupby() và agg()
print("\n6.2. Thống kê theo loại phòng:")
room_stats = final_merged.groupby('room_type').agg({
    'booking_id': 'count',
    'base_price': 'mean',
    'rating': 'mean'
}).round(2)

print(room_stats.to_string())

# ------------------------------------------------------------
# 7. PIVOT TABLE - Sử dụng pivot_table()
# ------------------------------------------------------------
print("\n" + "="*50)
print("7. PIVOT TABLE")
print("="*50)

# Tạo pivot table
booking_pivot = pd.pivot_table(
    final_merged,
    values='booking_id',
    index='city',
    columns='room_type',
    aggfunc='count',
    fill_value=0
)

print("7.1. Phân bố booking theo thành phố x loại phòng:")
print(booking_pivot.to_string())

# ------------------------------------------------------------
# 8. STACK/UNSTACK - Sử dụng stack() và unstack()
# ------------------------------------------------------------
print("\n" + "="*50)
print("8. STACK/UNSTACK")
print("="*50)

# Sử dụng stack()
stacked = booking_pivot.stack()
print("8.1. Stack pivot table:")
print(f"   Kiểu dữ liệu sau stack: {type(stacked)}")
print(f"   Shape: {stacked.shape}")

# Sử dụng unstack()
unstacked = stacked.unstack()
print("\n8.2. Unstack trở lại:")
print(f"   Kiểu dữ liệu sau unstack: {type(unstacked)}")
print(f"   Giống pivot gốc: {booking_pivot.equals(unstacked)}")

# ------------------------------------------------------------
# 9. XUẤT FILE KẾT QUẢ MERGE
# ------------------------------------------------------------
print("\n" + "="*50)
print("9. XUẤT FILE KẾT QUẢ MERGE")
print("="*50)

# Tạo thư mục task04_merge
task4_dir = BASE_DIR / "task04_merge"
task4_dir.mkdir(parents=True, exist_ok=True)

# Xuất file dữ liệu đã merge
output_file = OUTPUT_DIR / "complete_hotel_data.csv"
final_merged.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f" Đã xuất: {output_file}")

# Xuất file cho nhiệm vụ 4
task4_file = task4_dir / "merged_data.csv"
final_merged.to_csv(task4_file, index=False, encoding='utf-8-sig')
print(f" Đã xuất: {task4_file}")

# Xuất pivot table
booking_pivot.to_csv(task4_dir / "booking_pivot.csv", encoding='utf-8-sig')
print(f" Đã xuất: {task4_dir}/booking_pivot.csv")

# Xuất stacked data
stacked.to_csv(task4_dir / "stacked_data.csv", encoding='utf-8-sig')
print(f" Đã xuất: {task4_dir}/stacked_data.csv")

# Xuất booking không khớp
if len(missing_hotel_bookings) > 0:
    missing_hotel_bookings.to_csv(task4_dir / "missing_hotel_bookings.csv", index=False)
    print(f" Đã xuất: {task4_dir}/missing_hotel_bookings.csv")

# ------------------------------------------------------------
# 10. THỐNG KÊ KẾT QUẢ
# ------------------------------------------------------------
print("\n" + "="*60)
print("THỐNG KÊ KẾT QUẢ NHIỆM VỤ 4")
print("="*60)

print(f"\nHOTEL INFO:")
print(f"   - Số lượng khách sạn: {hotel_info_clean['hotel_id'].nunique()}")
print(f"   - Thành phố: {hotel_info_clean['city'].nunique()} thành phố")
print(f"   - Loại phòng: {hotel_info_clean['room_type'].nunique()} loại")
print(f"   - Giá trung bình: {hotel_info_clean['base_price'].mean():.2f}")

print(f"\nBOOKING RECORDS:")
print(f"   - Tổng số booking: {len(booking_records_clean)}")
print(f"   - Khoảng thời gian: {booking_records_clean['checkin_date'].min()} đến {booking_records_clean['checkin_date'].max()}")
print(f"   - Số đêm trung bình: {booking_records_clean['nights'].mean():.1f}")
print(f"   - Số khách trung bình: {booking_records_clean['num_guests'].mean():.1f}")

print(f"\nGUEST REVIEW:")
print(f"   - Tổng số review: {len(guest_review_clean)}")
print(f"   - Điểm đánh giá trung bình: {guest_review_clean['rating'].mean():.2f}/5")
print(f"   - Tỷ lệ có comment: {(guest_review_clean['comment'] != '').sum() / len(guest_review_clean) * 100:.1f}%")

print("\nTHÔNG TIN MERGE:")
print("-" * 40)
print(f"Tổng booking gốc: {len(booking_records)}")
print(f"Booking merge thành công: {len(final_merged)}")
print(f"Booking bị mất: {len(booking_records) - len(final_merged)}")
print(f"Tỷ lệ thành công: {(len(final_merged)/len(booking_records))*100:.1f}%")
print(f"Số khách sạn: {final_merged['hotel_id'].nunique()}")
print(f"Số thành phố: {final_merged['city'].nunique()}")
print(f"Booking có rating: {final_merged['rating'].notna().sum()}")
print(f"Booking không rating: {final_merged['rating'].isna().sum()}")

print("\nFILE ĐÃ TẠO:")
print("-" * 40)
print("data_clean/hotel_info_clean.csv")
print("data_clean/booking_records_clean.csv")
print("data_clean/guest_review_clean.csv")
print("data_clean/complete_hotel_data.csv")
print("task04_merge/merged_data.csv")
print("task04_merge/booking_pivot.csv")
print("task04_merge/stacked_data.csv")
if len(missing_hotel_bookings) > 0:
    print("task04_merge/missing_hotel_bookings.csv")

print("\nXEM TRƯỚC DỮ LIỆU ĐÃ MERGE (3 dòng đầu):")
print("-" * 70)
preview_cols = ['booking_id', 'hotel_id', 'hotel_name', 'city', 'room_type', 'checkin_date', 'rating']
print(final_merged[preview_cols].head(3).to_string())

print("\n" + "=" * 60)
print("NHIỆM VỤ 4 ĐÃ HOÀN THÀNH!")
print("=" * 60)

print("\nCÁC KỸ THUẬT ĐÃ SỬ DỤNG:")
print("1. read_csv() - Đọc dữ liệu")
print("2. str.strip(), str.upper(), str.title() - Làm sạch chuỗi")
print("3. to_datetime() - Chuẩn hóa ngày tháng")
print("4. astype(), pd.to_numeric() - Chuyển đổi kiểu dữ liệu")
print("5. fillna() - Xử lý giá trị thiếu")
print("6. Boolean Masking với isin() - Truy vấn dữ liệu")
print("7. loc - Truy vấn theo nhãn")
print("8. merge() với inner/left - Merge dữ liệu")
print("9. groupby(), agg() - Phân nhóm dữ liệu")
print("10. pivot_table() - Tạo bảng pivot")
print("11. stack(), unstack() - Chuyển đổi cấu trúc dữ liệu")
print("12. to_csv() - Xuất file kết quả")

NHIỆM VỤ 4: MERGE DỮ LIỆU KHÁCH SẠN

1. THIẾT LẬP ĐƯỜNG DẪN & NHẬP XUẤT FILE
--------------------------------------------------
Thư mục làm việc: c:\Users\Admin\OneDrive\Desktop\task04__merge\Task04_merge
Đọc dữ liệu từ: c:\Users\Admin\OneDrive\Desktop\task04__merge\Task04_merge
Xuất dữ liệu đến: c:\Users\Admin\OneDrive\Desktop\task04__merge\Task04_merge\data_clean
 File hotel_info.csv tồn tại
 File booking_records.csv tồn tại
 File guest_review.csv tồn tại

Đã đọc hotel_info.csv: 10 dòng, 5 cột
Đã đọc booking_records.csv: 120 dòng, 5 cột
Đã đọc guest_review.csv: 120 dòng, 3 cột

2. LÀM SẠCH DỮ LIỆU

2.1. Làm sạch hotel_info.csv
   Đã xử lý base_price: 0 giá trị NaN
   Đã làm sạch hotel_info

2.2. Làm sạch booking_records.csv
   Chuyển đổi checkin_date: 61 giá trị không hợp lệ
   Đã sửa 16 giá trị nights không hợp lệ
   Đã sửa 33 giá trị num_guests không hợp lệ
   Đã làm sạch booking_records

2.3. Làm sạch guest_review.csv
   Đã sửa 0 giá trị rating không hợp lệ
   Có 27 comment trống
